In [50]:
# auction image crawler v1_20211103
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import requests

## variables

item_url_set = set()
img_url_set = set() # 빈 집합 생성

## functions

def add_item_url():
    global item_url_set
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    a_tags = soup.select('a.link--itemcard')
    for a_tag in a_tags:
        item_url_set.add(a_tag['href'])

def add_img_url():
    global item_url_set
    global img_url_set
    for item_url in item_url_set:
        driver.get(item_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        img_tags = soup.select('div.box__viewer-container img')
        for img_tag in img_tags:
            img_url_set.add('http:' + img_tag['src'])

def download_img(img_folder, img_url):
    html_img = requests.get(img_url)
    imgFile = open(os.path.join(img_folder, os.path.basename(img_url)[:15] + '.jpg'), 'wb')        
    chunk_size = 1000000
    for chunk in html_img.iter_content(chunk_size):
        imgFile.write(chunk)
    imgFile.close()

def make_folder(keyword):
    img_folder = "C:\\Users\\2020\\crawling_images\\{}".format(keyword)
    if not os.path.exists(img_folder):
        os.makedirs(img_folder)
    return img_folder

def main(keyword, max_page):
    global item_url_set
    global img_url_set

    for page in range(1, max_page+1):

        item_url_set = set()
        img_url_set = set()

        driver.get('https://browse.auction.co.kr/search?keyword=' + keyword + '&k=32&p=' + str(page))
        time.sleep(5)

        add_item_url()
        print(str(page) + 'page의 모든 item_url을 얻었습니다.')
        time.sleep(3)
        img_folder = make_folder(keyword)

        add_img_url()
        print(str(page) + 'page까지 원본 img_url ' + str(len(img_url_set)) + '개를 얻었습니다.')
    
    img_url_set.discard('http://image.auction.co.kr/hanbando/202110/a3677c93-a39a-49ee-b2e5-aa9658e0e2c7.png')
    for img_url in img_url_set:
        download_img(img_folder, img_url)
        time.sleep(1)

    print(keyword + 'images {}개를 저장했습니다.'.format(len(img_url_set)))

## main
driver = webdriver.Chrome('C:\\Users\\2020\\chromedriver\\chromedriver.exe')
driver.set_window_size(1700, 1000)

if __name__ == "__main__":
    main('헤어+스프레이', 1)



1page의 모든 item_url을 얻었습니다.
1page의 모든 원본 img_url142개를 얻었습니다.
헤어+스프레이images 141개를 저장했습니다.
